In [1]:
#!/usr/bin/python

import numpy as np
import math
from astropy.io import fits as fits

from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm

import pandas as pd
from pandas import DataFrame, Series

from astropy.wcs import WCS as WCS

import import_ipynb
import Pipe3D_Asier_Test as Pipe3D

from mpdaf.obj import hms2deg
from mpdaf.obj import dms2deg

from astropy.convolution import Gaussian2DKernel
from astropy.convolution import convolve

from scipy.stats import ks_2samp
from scipy import stats

/home/asier/anaconda2/lib/python2.7/site-packages/pandas/_libs/__init__.py:4: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/home/asier/anaconda2/lib/python2.7/site-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/home/asier/anaconda2/lib/python2.7/site-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos, lib
/home/asier/anaconda2/lib/python2.7/site-packages/pandas/core/util/hashing.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import hashing, tslib
/home/asier/anaconda2/lib/python2.7/site-packages/pandas/core/indexes/base.py:7: Ru

importing Jupyter notebook from Pipe3D_Asier_Test.ipynb


/home/asier/anaconda2/lib/python2.7/site-packages/scipy/linalg/basic.py:17: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._solve_toeplitz import levinson
/home/asier/anaconda2/lib/python2.7/site-packages/scipy/linalg/__init__.py:207: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._decomp_update import *
/home/asier/anaconda2/lib/python2.7/site-packages/scipy/special/__init__.py:640: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._ufuncs import *
/home/asier/anaconda2/lib/python2.7/site-packages/scipy/special/_ellip_harm.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._ellip_harm_2 import _ellipsoid, _ellipsoid_norm
/home/asier/anaconda2/lib/python2.7/site-packages/scipy/signal/_max_len_seq.py:8: RuntimeWarning: numpy.dtype size changed, may ind

In [2]:
#Crear arrays con los datos de todas las galaxias.

#Numb_Lines = len(open('Names_Amusing_true.txt').readlines())
Numb_Lines = len(open('test.txt').readlines())

All_SN_type = np.zeros(Numb_Lines)
All_Prob_II = np.zeros(Numb_Lines)
All_Cumul_II = np.zeros(Numb_Lines)
All_Prob_Ia = np.zeros(Numb_Lines)
All_Cumul_Ia = np.zeros(Numb_Lines)

In [ ]:
#names = open('Names_Amusing_true.txt')
names = open('test.txt')
Galaxy = -1
nIa = 0
nII = 0
for lines in names:
    line= lines[:-1]
    print(line)
    Galaxy = Galaxy+1

    df = pd.read_csv('Amusing_Table.csv')  # To find the Galaxy data
    Names_1 = df['SN_name']
    Names_2 = df['def_name']
    R = 0

    for names in Names_1:              # Finding the Galaxy in the DataFrame
        R=R+1
        if line == names:
            Row = R-1
    R = 0
    for names in Names_2:              # Some Galaxies have the name in other column
        R=R+1
        if line == names:
            Row = R-1

    ## Pixel de la SN
    Pix_X = np.int(df.at[Row,'SNX'] -1)   # Enteros para que no de problemas al medir los mapas.
    Pix_Y = np.int(df.at[Row,'SNY'] -1)
    #
    ## Pixel del centro de la Galaxia
    Pix_X_GAL = int(df.at[Row,'GALX'] -1)
    Pix_Y_GAL = int(df.at[Row,'GALY'] -1)
    
    # Supernova type
    if df.at[Row,'SN_type'] == 'Ia':
        SN_TYPE = 1
        nIa = nIa + 1
    elif df.at[Row,'SN_type'] == 'II':
        SN_TYPE = 2
        nII = nII + 1
    else:
        SN_TYPE = 0
    
#~~~~~~~~~~~~ SN Rates ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

    # Call to Pipe3D code
    line = Pipe3D.Pipe3D_galaxy(line)
    
    # Obtain the maps of the galaxy
    # painting_galaxy_maps(self, stars_die, alfa, delta, sigma )
    alfa = 1
    delta = 0
    sigma = 0.1
    line.painting_galaxy_maps(False, alfa, delta, sigma)
    
    # Gaussian function
    stddev = 7
    gauss = Gaussian2DKernel(stddev)
    
    # Gaussian convolve
    line.SNII_rate_map = convolve(line.SNII_rate_map, gauss)
    line.SNIa_rate_map = convolve(line.SNIa_rate_map, gauss)
    
    # Eliminate negative values
    line.SNII_rate_map[line.SNII_rate_map<0.0] = 0.0
    line.SNIa_rate_map[line.SNIa_rate_map<0.0] = 0.0
    
# ------------- SN II ------------------------------------------------------------------------------------
    
    # An Array whit all the value of the SN II rate map
    line.Prob_List_II = np.ndarray.flatten(line.SNII_rate_map)
    # The array orther in value
    line.Order_Prob_List_II = np.sort(line.Prob_List_II)
    
    # Normalize
    line.Norm_SNII = 0
    for i in range(len(line.Order_Prob_List_II)):
        line.Norm_SNII = line.Norm_SNII + line.Order_Prob_List_II[i]

    line.Order_Prob_List_II = line.Order_Prob_List_II / line.Norm_SNII
    line.SNII_rate_map = line.SNII_rate_map / line.Norm_SNII
    
    # Calculate P
    line.cumulative_P_values_II = np.cumsum(line.Order_Prob_List_II)

    # Calculate P map
    line.cumulative_SNII_rate_map = np.interp(line.SNII_rate_map, line.Order_Prob_List_II, line.cumulative_P_values_II)
    
    # Measure Probability
    line.Prob_Val_SNII = 0
    line.Cumulative_Prob_SNII = 0
    square = np.arange(3)-1
    
    for j in square:
        for i in square:
            # The probability of SNII (p)
            line.Prob_Val_SNII = line.Prob_Val_SNII + line.SNII_rate_map[Pix_Y+j,Pix_X+i]
            # The cumulative probability of SNII (P)
            line.Cumulative_Prob_SNII = line.Cumulative_Prob_SNII + line.cumulative_SNII_rate_map[Pix_Y+j,Pix_X+i]
    
    line.Prob_Val_SNII = line.Prob_Val_SNII / 9
    line.Cumulative_Prob_SNII = line.Cumulative_Prob_SNII/9
    
# ------------- SN Ia ------------------------------------------------------------------------------------
    
    # An Array whit all the value of the SN Ia rate map
    line.Prob_List_Ia = np.ndarray.flatten(line.SNIa_rate_map)
    # The array orther in value
    line.Order_Prob_List_Ia = np.sort(line.Prob_List_Ia)
    
    # Normalize
    line.Norm_SNIa = 0
    for i in range(len(line.Order_Prob_List_Ia)):
        line.Norm_SNIa = line.Norm_SNIa + line.Order_Prob_List_Ia[i]

    line.Order_Prob_List_Ia = line.Order_Prob_List_Ia / line.Norm_SNIa
    line.SNIa_rate_map = line.SNIa_rate_map / line.Norm_SNIa
    
    # Calculate P
    line.cumulative_P_values_Ia = np.cumsum(line.Order_Prob_List_Ia)

    # Calculate P map
    line.cumulative_SNIa_rate_map = np.interp(line.SNIa_rate_map, line.Order_Prob_List_Ia, line.cumulative_P_values_Ia)
    
    # Measure Probability
    line.Prob_Val_SNIa = 0
    line.Cumulative_Prob_SNIa = 0
    square = np.arange(3)-1
    
    for j in square:
        for i in square:
            # The probability of SNIa (p)
            line.Prob_Val_SNIa = line.Prob_Val_SNIa + line.SNIa_rate_map[Pix_Y+j,Pix_X+i]
            # The cumulative probability of SNIa (P)
            line.Cumulative_Prob_SNIa = line.Cumulative_Prob_SNIa + line.cumulative_SNIa_rate_map[Pix_Y+j,Pix_X+i]
    
    line.Prob_Val_SNIa = line.Prob_Val_SNIa / 9
    line.Cumulative_Prob_SNIa = line.Cumulative_Prob_SNIa/9
    
# ------------- Save Data ------------------------------------------------------------------------------------
    
    # SN type
    All_SN_type[Galaxy] = SN_TYPE
    # SN II and cumulative SN II Probability
    All_Prob_II[Galaxy] = line.Prob_Val_SNII
    All_Cumul_II[Galaxy] = line.Cumulative_Prob_SNII
    # SN Ia and cumulative SN Ia Probability
    All_Prob_Ia[Galaxy] = line.Prob_Val_SNIa
    All_Cumul_Ia[Galaxy] = line.Cumulative_Prob_SNIa
    
    
    
    
    data_CC = np.column_stack((All_SN_type,All_Prob_II,All_Cumul_II))
    np.savetxt('../Results_CC_'+str(sigma)+'',data_CC)
    
    data_Ia = np.column_stack((All_SN_type,All_Prob_Ia,All_Cumul_Ia))
    np.savetxt('../Results_Ia_'+str(alfa)+'_'+str(delta)+'',data_Ia)

ASASSN13an
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
